In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
pathh = 'Data/tweets/PrelabelData.csv'
df1 = pd.read_csv(pathh)
df1 = df1.dropna()
n = 500
df = df1.iloc[:n]
df_test = df1.iloc[n:600]
df = df[['text','sentiment']]

def model_func():
  text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preproc = hub.KerasLayer(("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"), name='preprocessing')
  encoder = hub.KerasLayer(("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"), trainable=True, name='BERT_encoder')
  to_output = encoder(preproc(text))
  model_network = to_output['pooled_output']
  model_network = tf.keras.layers.Dropout(0.1)(model_network)
  model_network = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(model_network)
  return tf.keras.Model(text, model_network)
the_model = model_func()


In [ ]:
train_dir = 'Data/tweets/datatr/'
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 123

train_data = tf.keras.utils.text_dataset_from_directory(
    'Data/tweets/datatr/train/',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)
class_names = train_data.class_names
train_tuned = train_data.cache().prefetch(buffer_size=AUTOTUNE)

val_data = tf.keras.utils.text_dataset_from_directory(
    'Data/tweets/datatr/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)
val_tuned = val_data.cache().prefetch(buffer_size=AUTOTUNE)

test_data = tf.keras.utils.text_dataset_from_directory(
    'Data/tweets/datatr/train',
    batch_size=batch_size)
test_tuned = test_data.cache().prefetch(buffer_size=AUTOTUNE)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.keras.metrics.SparseCategoricalAccuracy()
epochs = 7


init_lr = 3e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=init_lr)

the_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

history = the_model.fit(x=train_tuned,
                               validation_data=test_tuned,
                               epochs=epochs)


In [ ]:
the_model.save('my_model2', include_optimizer=False)